- 웹 통신 세 가지 방식
    
    1. 표준 통신 - 링크클릭, 주소 입력 -> 웹 브라우저 화면 전체를 갱신 => 소스 보기를 하면 나옴
    2. 비동기 통신 - 병렬 처리 -> 일부를 갱신 - Ajax통신 => 소스 보기로 안 나옴 / Ajax통신에 사용하는 주소를 파악해서 직접 요청
    3. 웹 소켓

- 페이지 소스 보기를 했을 때 안나옴 -> 프레임 소스 보기가 없음 -> 아작스인가..?
- F12에서 network를 누르면 Type에 xhr => 비동기 통신이라는 것을 알 수 있음 => Name 링크를 더블클릭하면 Ajax통신에 사용하는 주소를 파악 

In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from urllib.request import urlopen
from IPython.display import clear_output

- 소스 보기 시 데이터가 없었다.
- iframe도 없다.
- 비동기 통신인지 확인해야 한다.
- 크롬에서 F12를 눌러 개발자 도구를 띄우고 네트워크 탭을 띄워준다.
- 뭔가 변화를 주고(다른 것을 클릭해봄) 네트워크를 확인했을 때 type이 xhr이라고 나온다면 비동기 통신이다.
- 네트워크의 탭에 나와있는 주소를 더블클릭해서 브라우저에서 주소를 확인한다.

In [2]:
# 요청 함수
def getSource(site) :
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers = header_info)
    # print(response.text)
    
    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

In [3]:
# 하루의 경기결과를 가져오는 함수
def getData(soup):
    
    # 데이터를 담은 딕셔너리
    data_dict = {
        '팀1이름' : [],
        '팀2이름' : [],
        '팀1투수' : [],
        '팀2투수' : [],
        '팀1점수' : [],
        '팀2점수' : []
    }
    
    # 경기 결과 전체를 가져온다
    a1 = soup.select_one('#todaySchedule')
    # print(a1)
    
    # li 태그들을 가져온다
    a2 = a1.select('li.end')
    # print(a2)

    # li 태그의 수만큼 반복한다
    for a3 in a2:
        # 좌측 팀 이름을 가져온다
        a4 = a3.select_one('div.vs_lft > p > strong')
        data1 = a4.text.strip()
        # print(data1)
        
        # 우측 팀 이름을 가져온다
        a5 = a3.select_one('div.vs_rgt > p > strong')
        data2 = a5.text.strip()
        # print(data2)
        
        # 좌측 팀 투수이름을 가져온다
        a6 = a3.select_one('div.vs_lft > p > span > a')
        
        if a6 != None:
            data3 = a6.text.strip()
        else:
            data3 = np.nan
        # print(data3)        

        # 우측 팀 투수이름을 가져온다
        a7 = a3.select_one('div.vs_rgt > p > span > a')
        
        if a7 != None:
            data4 = a7.text.strip()
        else:
            data4 = np.nan
        # print(data4)             
        
        # 좌측 팀 점수를 가져온다
        a8 = a3.select_one('div.vs_lft > strong')
        data5 = a8.text.strip()
        # print(data5)        
        
        # 우측 팀 점수를 가져온다
        a9 = a3.select_one('div.vs_rgt > strong')
        data6 = a9.text.strip()
        # print(data6)            
        
        # print(data1)   
        # print(data2)  
        # print(data3)  
        # print(data4)  
        # print(data5)  
        # print(data6)  
        
        # 딕셔너리에 데이터를 담는다
        data_dict['팀1이름'].append(data1)
        data_dict['팀2이름'].append(data2)
        data_dict['팀1투수'].append(data3)
        data_dict['팀2투수'].append(data4)
        data_dict['팀1점수'].append(data5)
        data_dict['팀2점수'].append(data6)

    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    if os.path.exists('data4.csv') == False:
        # 파일이 없을 경우
        df1.to_csv('data4.csv', encoding='utf-8-sig', index=False) # encoding='utf-8-sig' : 엑셀로 열 때 한글이 안깨지도록
    else:
        df1.to_csv('data4.csv', encoding='utf-8-sig', index=False, header=False, mode='a') # mode='a' 기존의 것에 붙여줌        

In [4]:
# 다음 날짜 값을 가지고 온다
def getNext(soup):
    a1 = soup.select_one('body > div > ul > li:nth-child(6) > a')
    # print(a1)
    
    # onclick 속성 값을 가져온다
    a2 = a1.attrs['onclick']
    # print(a2)
    
    # 날짜 값을 가져온다
    a3 = a2.split("'")
    # print(a3)
    
    return a3[1]

In [5]:
now = '20190312'

while True:
    time.sleep(1)
    clear_output(wait=True)
    
    site = f'https://sports.news.naver.com/schedule/scoreBoard.nhn?date={now}&category=kbo'
    
    print(now)

    soup = getSource(site)
    getData(soup)
    now = getNext(soup)
    
    if now[:4] == '2020':
        print('수집완료')
        break

20191026


,팀1이름,팀2이름,팀1투수,팀2투수,팀1점수,팀2점수
0,두산,키움,이용찬,브리검,11,9


수집완료
